NOMBRES ETC

- Explicitar tipo de problema (clasificación / regresión)
- Definición, justificación de uso de métricas para medir el desempeño

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import statsmodels.api as sm

# FEATURE ENGINE
from feature_engine.wrappers import SklearnTransformerWrapper
from feature_engine.imputation import CategoricalImputer, MeanMedianImputer
#from feature_engine.encoding import OrdinalEncoder

# SKLEARN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score, mean_squared_error
from sklearn.model_selection import train_test_split, KFold, LeaveOneOut, cross_val_score, cross_validate

# FACTOR ANALYZER
from factor_analyzer import FactorAnalyzer

# OTROS
import helpers as hp
from IPython.display import HTML 


### Binarias
- `school`: Escuela del estudiante. (binaria: 'GP' - Gabriel Pereira o 'MS' - Mousinho da Silveira).
- `sex`: Sexo del estudiante. (binaria: 'F' - Mujer o 'M' - Hombre).
- `address`: Ubicación de la casa del estudiante. (binaria: 'U' - urbana o 'R' - rural).
- `famsize`: Tamaño de la familia. (binaria: 'LE3' - less or equal to 3 or 'GT3' - greater than 3).
- `Pstatus`: Estado cohabitacional de los padres. (binaria: 'T' - cohabitando juntos o 'A' - viviendo separados).
- `schoolsup`: Apoyo educacional del colegio. (binaria: si o no).
- `famsup`: Apoyo educacional familiar. (binaria: si o no).
- `paid`: Clases particulares pagadas (matemáticas o portugués) (binaria: sí o no).
- `activities`: Actividades extracurriculares. (binaria: si o no).
- `nursery`: Asistió a guardería infantil. (binaria: si o no).
- `higher`: Desea proseguir estudios superiores (binaria: si o no).
- `internet`: Acceso a internet desde el hogar (binaria: si o no).
- `romantic`: Relación romántica (binaria: si o no).

### Categóricas (Nominales)
- `Mjob`: Ocupación de la madre. (nominal: 'teacher' profesora, 'health' relacionada a salud, 'services' (e.g. administración pública o policía), 'at_home' en casa u 'other' otra).
- `Fjob`: Ocupación del padre (nominal: 'teacher' profesor, 'health' relacionado a salud, 'services' (e.g. administración pública o policía), 'at_home' en casa u 'other' otra).
- `reason`: Razón para escoger la escuela (nominal: 'home' cercano a casa, 'reputation' reputación de la escuela, 'course' preferencia de cursos u 'other' otra).
- `guardian`: Apoderado del estudiante (nominal: 'mother' madre, 'father' padre u 'other' otro).

### Numéricas
- `age`: Edad del estudiante. (numérica: de 15 a 22).
- `Medu`: Nivel educacional de la madre. (numérica: 0 - ninguno, 1 - educación básica (4to), 2 - de 5to a 9, 3 - educación media, o 4 - educación superior).
- `Fedu`: Nivel educacional del padre. (numérica: 0 - ninguno, 1 - educación básica (4to), 2 - de 5to a 9, 3 - educación media, o 4 - educación superior).
- `failures`: Número de clases reprobadas. (numérica: n si 1<=n<3, de lo contrario 4).
- `famrel`: Calidad de las relaciones familiares. (numérica: de 1 - muy malas a 5 - excelentes).
- `freetime`: Tiempo libre fuera del colegio (numérica: de 1 - muy poco a 5 - mucho).
- `goout`: Salidas con amigos (numérica: de 1 - muy pocas a 5 - muchas).
- `Dalc`: Consumo de alcohol en día de semana (numérica: de 1 - muy bajo a 5 - muy alto).
- `Walc`: Consumo de alcohol en fines de semana (numérica: de 1 - muy bajo a 5 - muy alto).
- `health`: Estado de salud actual (numérica: from 1 - muy malo to 5 - muy bueno).
- `absences`: Cantidad de ausencias escolares (numérica: de 0 a 93).
- `traveltime`: Tiempo de viaje entre hogar y colegio. Se debe codificar como:
	* 1 si es menos de 15 min,
	* 2 si es de 15 a 30 min,
	* 3 si es de 30 min. a 1 hora,
	* 4 - si es más de 1 hora).
- `studytime`: Horas semanales dedicadas al estudio. Se debe codificar como:
	* 1 si es menos de 2 horas,
	* 2 si es de 2 a 5 horas,
	* 3 si es de 5 a 10 horas,
	* 4 si es más de 10 horas.

### Target (Numérico)
- `G1`: Notas durante el primer semestre (numérica: de 0 a 20). Este es uno de sus vectores objetivos para el modelo descriptivo.
- `G2`: Notas durante el segundo semestre (numérica: de 0 a 20). Este es uno de sus vectores objetivos para el modelo descriptivo.
- `G3`: Promedio final (numérica: de 0 a 20). Este es uno de sus vectores objetivos para el modelo descriptivo y el vector a predecir en el modelo predictivo.

### Resolución de aspectos adicionales a considerar:
### Parte 1: 
 - Importar correctamente la data con separador indicado
 - Recodificación de valores nulos
 - Reasignación y limpieza de 3 variables númericas ingresadas como string("age", "goout", "health)
 - No esta indicado, pero se imputa los nulos en variables categóricas por moda y para variables númericas por la media.

In [ ]:
def load_students():
        df = pd.read_csv('students.csv',sep='|',index_col = 'Unnamed: 0', na_values=['nulidade', 'sem validade', 'zero'])
        df.age=df.age.str.replace('"','')
        df.goout=df.goout.str.replace('"','')
        df.health=df.health.str.replace('"','')
        #df.astype({'age': 'float', 'goout':'float','health':'float'})
        df.age = pd.to_numeric(df.age, errors='coerce')
        df.goout = pd.to_numeric(df.goout, errors='coerce')
        df.health = pd.to_numeric(df.health, errors='coerce')
        return df

In [ ]:
df = load_students()
df

In [ ]:
hp.describe_variables(df)

#### Modelo Explicativo:

In [ ]:
# Definición de dataframes eliminando G1, G2 y G3 según corresponda
dfs = {}
dfs['G1'] = df.drop(["G2", "G3"], axis=1)
dfs['G2'] = df.drop(["G1", "G3"], axis=1)
dfs['G3'] = df.drop(["G1", "G2"], axis=1)


In [ ]:
varremove = {}
varremove['G1'] = ["traveltime", "Pstatus", "age", "nursery", "famrel", "school", "Dalc", "activities", "absences", "Medu", "romantic", "internet", "guardian", "paid", "address", "reason", "Walc", "famsize", "health", "freetime", "Fedu"]
varremove['G2'] = ["absences","Fedu","nursery","activities","Dalc","Walc","famrel","paid","freetime","Pstatus","Mjob","school","health","age","guardian","traveltime","Fjob","higher","internet","famsize","famsup"]
varremove['G3'] = ["nursery", "absences", "Mjob", "health", "Fedu", "guardian", "paid", "traveltime", "Pstatus", "famrel", "age", "school", "address", "Walc", "Dalc", "freetime", "activities", "studytime", "internet", "famsup", "Fjob", "famsize", "higher", "schoolsup"]
variables = dfs['G1'].drop(columns='G1').columns.to_list()
dicc = {key:[(col in varremove[key]) for col in variables] for key in varremove.keys()}
test_df = pd.DataFrame(dicc,index = variables)
filtro = (~(test_df.all(axis='columns') | ~test_df.any(axis='columns')))
test_df[filtro]

Variables Sacadas en G1 que permanecen en G2, G3
- Medu

Variables Sacadas en G2 y G3 que no se sacaron en G1
- famsize
- Fedu
- Mjob
- Fjob
- guardian
- famsup
- paid
- higher
- internet
- freetime
- Walc
- health

Variables 
- studytime
- schoolsup
- address

In [ ]:
dfs['G1_r'] = dfs["G1"].drop(columns=varremove["G1"])
dfs['G2_r'] = dfs["G2"].drop(columns=varremove["G2"])
dfs['G3_r'] = dfs["G3"].drop(columns=varremove["G3"])

In [ ]:
targets = list(dfs.keys())
df_t = {}
pipes = {}
X_f = {}
ys = {}
x1 = {}
models = {}
targetsname = {'G1_r': "G1",'G2_r': "G2", 'G3_r': "G3", "G1": "G1", "G2": "G2", "G3":"G3"}

for tname in targets:
    df_t[tname] = dfs[tname].dropna(axis=0, subset= [targetsname[tname]])
    x1[tname] = df_t[tname].drop(columns=[targetsname[tname]])
    ys[tname] = df_t[tname][targetsname[tname]]
    if tname in ['G1', 'G2', 'G3']:
        sns.displot(ys[tname])
        plt.title(tname)
        plt.show(f'Target: {targetsname[tname]}')
    


In [ ]:
for tname in targets:
    
    # Grupos de variables    
    x1_cat = x1[tname].select_dtypes(np.object_).columns.to_list()                                                         # Categoricas
    x1_catbin = [col for col in x1[tname].select_dtypes(np.object_).columns.to_list() if len(x1[tname][col].value_counts()) == 2] # Categoricas binarias
    x1_catnum = [col for col in x1[tname].select_dtypes(np.object_).columns.to_list() if len(x1[tname][col].value_counts()) > 2]  # Categoricas numericas
    x1_num = x1[tname].select_dtypes(np.number).columns.to_list()                                                          # Numericas
    
    # Definición de pipelines
    pipes[tname] = Pipeline(steps = [
                    ("mmi" , MeanMedianImputer(imputation_method='median')),
                    ("ci" , CategoricalImputer(imputation_method='frequent')),
                    ('oe', SklearnTransformerWrapper(OrdinalEncoder(categories=hp.OrdinalEncoderListCategories(x1[tname][x1_catbin], direction='ascending', bin_or_num='bin') + hp.OrdinalEncoderListCategories(x1[tname][x1_catnum], direction='descending', bin_or_num='num') ),variables =  x1_catbin + x1_catnum)),
                    ("sc" , SklearnTransformerWrapper(StandardScaler(), variables= x1_num))
                    ])
    # Ajuste
    X_f[tname] = pipes[tname].fit_transform(x1[tname],ys[tname]) # Transformo data de acuerdo al pipeline
    
    # Test para F-Analysis
    if tname == "G1":
        hp.test_factor_analyzer(X_f[tname].loc[:,"famrel":"health"].values) 
        fa = FactorAnalyzer()
        fa.fit(X_f[tname].loc[:,"famrel":"health"])
        ev, v = fa.get_eigenvalues() 
        ev.shape
        plt.plot(range(1,ev.shape[0]+1), ev, "-o")
        plt.axhline(y=1,c="red",ls="--")
        plt.title("Scree Plot")
        plt.show();

    
    # Modelo descriptivo
    X_f[tname] = sm.add_constant(X_f[tname]) # Agrego una columna de 1's para el modelo de regresión lineal
    models[tname] = sm.OLS(ys[tname] ,X_f[tname]).fit() # Modelo de regresión lineal
    predsg1 = models[tname].predict(X_f[tname]) # Predicciones del modelo de regresión lineal

    # Prints
    display(HTML(f'<p style="padding:20px 30px; background:tomato;font-size:20px;color: white">Modelo exploratorio para {tname}</p>'))
    display(models[tname].summary())
    hp.evaluation(models[tname], ys[tname], predsg1)




#### Modelo Predictivo  (G3)

In [12]:
#Modelo usando todas las variables (menos G1 y G2):

cvs   = {}
scorings = {"r2":"r2", "neg_root_mean_squared_error": "neg_root_mean_squared_error"}
metrics = scorings.keys()

for tname in X_f.keys():

    X_train, X_test, y_train, y_test = train_test_split (X_f[tname], ys[tname], test_size=0.2, random_state=42)
    lr = LinearRegression()
    lr.fit(X_train,y_train)

    y_pred = lr.predict(X_test)
    cv = KFold(n_splits=2, random_state=42, shuffle=True) #LeaveOneOut()
    cvs[tname] = cross_validate(lr, X_f[tname], ys[tname], cv=cv, scoring = scorings)

    
    print(f"\nModelo {tname}: ")
    print("R2:", r2_score (y_test,y_pred))
    print("RMSE:", mean_squared_error(y_test,y_pred,squared=False))


Modelo G1: 
R2: 0.21782856553842733
RMSE: 3.14649057238124

Modelo G2: 
R2: 0.21596685924634706
RMSE: 3.341242024651787

Modelo G3: 
R2: 0.03775471315120604
RMSE: 4.547224395899578

Modelo G1_r: 
R2: 0.2308173655463357
RMSE: 3.12025576787068

Modelo G2_r: 
R2: 0.3059197800205178
RMSE: 3.1437323118696967

Modelo G3_r: 
R2: 0.12343076687985255
RMSE: 4.340068674454519


In [13]:
table_str = '<table><tr><th>Models</th><th>' + '</th><th>'.join(metrics) + '</th><th>Nº de variables</th><th>fit time</th><th>score time</th></tr>'

for tname in X_f.keys():
    table_str += '<tr>'
    table_str += f"<td>{tname}</td>"
    for metric in metrics:
        table_str += f"<td>{cvs[tname]['test_'+metric].mean():.4f}</td>"
    table_str += f"<td>{len(X_f[tname].columns.to_list())}</td>"
    table_str += f"<td>{cvs[tname]['fit_time'].mean():.4f}</td>"
    table_str += f"<td>{cvs[tname]['score_time'].mean():.4f}</td>"
    table_str += '</tr>'

display(HTML(table_str))

Models,r2,neg_root_mean_squared_error,Nº de variables,fit time,score time
G1,0.0437,-3.2375,31,0.0033,0.0018
G2,0.0481,-3.6596,31,0.0040,0.0025
G3,0.0624,-4.4103,31,0.0022,0.0020
G1_r,0.1956,-2.9718,10,0.0026,0.0013
G2_r,0.1828,-3.3910,10,0.0017,0.0012
G3_r,0.1376,-4.2283,7,0.0016,0.0018
